In [1]:
# import the libraries

import pandas as pd
from pandas_datareader import data as pdr
import yfinance as yf
import statsmodels.api as sm
import numpy as np
from pandas_datareader.famafrench import get_available_datasets

In [2]:
get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [3]:
# Read in three factors (market, size, book to market) from Professor French's website as a dataframe
ff = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start = '1991-1-1')[0]

ff.head()

/var/folders/rx/wfm78crj6bg1zsfg6pssdc6r0000gn/T/ipykernel_78494/1019251418.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start = '1991-1-1')[0]


,Mkt-RF,SMB,HML,RF
Date,,,,
1991-01-02,-0.95,0.64,0.82,0.023
1991-01-03,-1.25,0.28,1.17,0.023
1991-01-04,-0.24,0.12,0.42,0.023
1991-01-07,-1.72,0.32,0.23,0.023
1991-01-08,-0.29,-0.36,-0.01,0.023


In [4]:
# Read in the data of Intel, AMD, and NVIDIA

yf.pdr_override()

tickers = ['INTL', 'AMD', 'NVDA']

mydata = pd.DataFrame()

for t in tickers:
    mydata[t] = pdr.get_data_yahoo(t)['Adj Close']

mydata.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,INTL,AMD,NVDA
Date,,,
2022-12-02,19.344280,74.980003,168.677231
2022-12-05,19.087631,73.620003,166.018555
2022-12-06,19.075182,70.269997,159.791595
2022-12-07,19.020597,70.139999,161.120941
2022-12-08,19.179567,70.470001,171.605804


In [5]:
# Calculate the simple daily returns using Adj. Close

mydata_returns = (mydata / mydata.shift(1) - 1) * 100 

mydata_returns.head()

,INTL,AMD,NVDA
Date,,,
2022-12-02,NaN,NaN,NaN
2022-12-05,-1.326744,-1.813818,-1.576191
2022-12-06,-0.065222,-4.550402,-3.750761
2022-12-07,-0.286155,-0.184997,0.831925
2022-12-08,0.835778,0.470490,6.507449


In [6]:
#Merge the factors with stock returns

all = pd.merge(mydata_returns, ff, left_index = True, right_index = True)
all.head()

,INTL,AMD,NVDA,Mkt-RF,SMB,HML,RF
Date,,,,,,,
2022-12-02,NaN,NaN,NaN,-0.08,0.92,-0.55,0.016
2022-12-05,-1.326744,-1.813818,-1.576191,-2.02,-0.59,-0.51,0.016
2022-12-06,-0.065222,-4.550402,-3.750761,-1.51,-0.24,1.16,0.016
2022-12-07,-0.286155,-0.184997,0.831925,-0.24,-0.13,-0.34,0.016
2022-12-08,0.835778,0.470490,6.507449,0.77,0.20,-1.22,0.016


In [7]:
all = all.dropna()
all.head()

,INTL,AMD,NVDA,Mkt-RF,SMB,HML,RF
Date,,,,,,,
2022-12-05,-1.326744,-1.813818,-1.576191,-2.02,-0.59,-0.51,0.016
2022-12-06,-0.065222,-4.550402,-3.750761,-1.51,-0.24,1.16,0.016
2022-12-07,-0.286155,-0.184997,0.831925,-0.24,-0.13,-0.34,0.016
2022-12-08,0.835778,0.470490,6.507449,0.77,0.20,-1.22,0.016
2022-12-09,-0.384472,-2.667809,-0.978521,-0.80,-0.55,0.36,0.016


In [8]:
# Calculate excess stocks returns (stock return - rf)

all['INTL - RF'] = all['INTL'] - all['RF']
all['AMD - RF'] = all['AMD'] - all['RF']
all['NVDA - RF'] = all['NVDA'] - all['RF']
all.head()

,INTL,AMD,NVDA,Mkt-RF,SMB,HML,RF,INTL - RF,AMD - RF,NVDA - RF
Date,,,,,,,,,,
2022-12-05,-1.326744,-1.813818,-1.576191,-2.02,-0.59,-0.51,0.016,-1.342744,-1.829818,-1.592191
2022-12-06,-0.065222,-4.550402,-3.750761,-1.51,-0.24,1.16,0.016,-0.081222,-4.566402,-3.766761
2022-12-07,-0.286155,-0.184997,0.831925,-0.24,-0.13,-0.34,0.016,-0.302155,-0.200997,0.815925
2022-12-08,0.835778,0.470490,6.507449,0.77,0.20,-1.22,0.016,0.819778,0.454490,6.491449
2022-12-09,-0.384472,-2.667809,-0.978521,-0.80,-0.55,0.36,0.016,-0.400472,-2.683809,-0.994521
